In [4]:
import cv2
import numpy as np
import pyrealsense2 as rs

WIDTH = 848
HEIGHT = 480
FPS = 60
THRESHOLD_MAX = 0.6#これより近い距離の画素を無視する
THRESHOLD_MIN = 1.0#これより遠い距離の画素を無視する

def red_detect(img):
    # HSV色空間に変換
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 赤色のHSVの値域1
    hsv_min = np.array([0,127,0])
    hsv_max = np.array([30,255,255])
    mask1 = cv2.inRange(hsv, hsv_min, hsv_max)

    # 赤色のHSVの値域2
    hsv_min = np.array([150,127,0])
    hsv_max = np.array([179,255,255])
    mask2 = cv2.inRange(hsv, hsv_min, hsv_max)
    
    return mask2

# ブロブ解析
def analysis_blob(binary_img):
    # 2値画像のラベリング処理
    label = cv2.connectedComponentsWithStats(binary_img)

    # ブロブ情報を項目別に抽出
    nlabels = label[0] - 1
    data = np.delete(label[2], 0, 0)
    center = np.delete(label[3], 0, 0)
    
    if np.any(data)==True:
        # ブロブ面積最大のインデックス
        max_index = np.argmax(data[:, 3])

        # 面積最大ブロブの情報格納用
        maxblob = {}

        # 面積最大ブロブの各種情報を取得
        maxblob["upper_left"] = (data[:, 0][max_index], data[:, 1][max_index]) # 左上座標
        maxblob["width"] = data[:, 2][max_index]  # 幅
        maxblob["height"] = data[:, 3][max_index]  # 高さ
        maxblob["area"] = data[:, 4][max_index]   # 面積
        maxblob["center"] = center[max_index]  # 中心座標
    else:
        maxblob=0
    return maxblob

def main():

    # ストリーム(Depth/Color)の設定
    config = rs.config()
    config.enable_stream(rs.stream.color, WIDTH, HEIGHT, rs.format.bgr8, FPS)
    config.enable_stream(rs.stream.depth, WIDTH, HEIGHT, rs.format.z16, FPS)

    # ストリーミング開始
    pipeline = rs.pipeline()
    profile = pipeline.start(config)

    # Alignオブジェクト生成
    align_to = rs.stream.color
    align = rs.align(align_to)

    #画像に合わせたdepth指定の値算出
    depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()
    max_dist = THRESHOLD_MAX/depth_scale
    min_dist = THRESHOLD_MIN/depth_scale

    try:
        while True:
            #フレーム待ち(Color & Depth)
            frames = pipeline.wait_for_frames()

            #画角補正した情報フレーム取得
            aligned_frames = align.process(frames)
            color_frame = aligned_frames.get_color_frame()
            depth_frame = aligned_frames.get_depth_frame()
            if not depth_frame or not color_frame:
                continue

            #RGB画像をnumpy arrayに
            color_image = np.asanyarray(color_frame.get_data())

            #depth画像をnumpy arrayに
            # depth_color_frame = rs.colorizer().colorize(depth_frame)
            # depth_color_image = np.asanyarray(depth_color_frame.get_data())

            #指定距離最大・最小以上をカットしたdepth画像
            depth_image = np.asanyarray(depth_frame.get_data())
            depth_filterd_image = ((min_dist < depth_image) ^ (max_dist < depth_image)) * depth_image
            # depth_gray_filtered_image = (depth_filterd_image * 255.0 / m)

            #指定距離最大・最小以上をカットしたRGB画像
            color_filtered_image = (depth_filterd_image.reshape((HEIGHT, WIDTH, 1)) > 0) * color_image



            # カラートラッキング（赤色）
            mask = red_detect(color_filtered_image)

            # マスク画像をブロブ解析（面積最大のブロブ情報を取得）
            target = analysis_blob(mask)

            if not target==0:
                # 面積最大ブロブの中心座標を取得
                center_x = int(target["center"][0])
                center_y = int(target["center"][1])

                # フレームに面積最大ブロブの中心周囲を円で描く
                cv2.circle(color_filtered_image, (center_x, center_y), 30, (0, 200, 0),
                        thickness=3, lineType=cv2.LINE_AA)

                depth = depth_frame.get_distance(center_x, center_y)

                # ウィンドウ表示
                cv2.putText(color_filtered_image, str(depth)+"m", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), thickness=2)
                cv2.imshow("Frame", color_filtered_image)
                cv2.imshow("Mask", mask)
            

            if cv2.waitKey(1) & 0xFF == ord('q'):
                # When everything done, release the capture
                break
    finally:
        pipeline.stop()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()